# Ingest Data used for RAG Application

In [2]:
from domino_data.vectordb import DominoPineconeConfiguration

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Qdrant

import csv
import os
import pinecone
import sys

from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

## Ingest relevant FAQ & support data scraped from Domino's help site

https://tickets.dominodatalab.com/hc/en-us/sections/360000203463-FAQ

In [3]:
csv.field_size_limit(sys.maxsize)

loader = CSVLoader(file_path="/mnt/data/{}/domino_help_clean.csv".format(os.environ['DOMINO_PROJECT_NAME']), source_column="url")

data = loader.load()

In [4]:
article_texts = []
chunk_size=1000
chunk_overlap=200
strip_whitespace = True

article_text = loader.load_and_split(
        RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap, 
            strip_whitespace=strip_whitespace
        )
    )
    
# refine texts
for chunk in article_text:
    # index of documentation path 
    path_idx = chunk.page_content.find("\n")
    chunk.page_content = chunk.page_content[path_idx + 1:]
        
article_texts.append(article_text)

In [5]:
# Print sample to test quality
article_texts[0][20].page_content

'\nTo resume your upload use the following syntax:\n\n**% domino upload-dataset user\\_id/project\\_name/upload\\_dataset Users/myuser/data aed75b88-cb1c-4ce3-a06b-05b675c296cf**\n\n### Managing Path Collision\n\nUse the`--fileUploadSetting`option to handle path collisions as follows:\n\n1.  `overwrite`: If a file already exists in the Dataset, the new file overwrites the existing file.\n    \n2.  `rename`: If a file already exists in the Dataset, the new file is uploaded and renamed with`_1`appended to the filename. For example,`/Users/myUser/data/file.txt`becomes`/Users/myUser/data/file_1.txt`\n    \n3.  `ignore`: If a file already exists in the Dataset, the new file is ignored.\n    \n\nTo use this option use the following syntax:'

### Load an embedding model and store the embeddings.

This example loads the "BAAI/bge-small-en" embedding model from HuggingFace, but approved embeddingcs can be stored in a Domino Dataset or your managed storage.

The embedigs are saved to the Project's Domino Dataset, but could also be stored in an attached data source.

In [6]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/{}/model_cache/'.format(os.environ['DOMINO_PROJECT_NAME'])
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

/home/domino/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Save the embeddings to a Vector Database

In [7]:
datasource_name = "pinecone-domino-support"
conf = DominoPineconeConfiguration(datasource=datasource_name)

print(pinecone.__version__)

# The pinecone API key should be provided when creating the Domino Data Source and persisted securely.
# This api_key variable here is only used for satisfying the native pinecone python client initialization where
# api_key is a mandatory non-empty field.

api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

pinecone.init(
    api_key=api_key,
    environment="domino",
    openapi_config=conf)

3.2.2


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
print(pinecone.list_indexes())
print(pinecone.describe_index('domino-support'))

In [ ]:
# check if index already exists, if not we create it
index_name = "domino-support"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=384, ## 384 for bge-small-en 
        metric='cosine'
    )

#insert the embeddings
from langchain.vectorstores import Pinecone

vector_store = Pinecone.from_documents(
    article_texts[0],
    embeddings,
    index_name=index_name,
    namespace='domino-help'
)

In [ ]:
# Uncomment below if you want to use Qdrant without the AI g/w
# qdrant_key = os.environ.get("QDRANT_KEY")
# qdrant_url = "https://59f8f159-fb60-44e8-bfc4-9f35c77ca8d4.us-east4-0.gcp.cloud.qdrant.io:6333"

# doc_store = Qdrant.from_documents(article_texts[0],
#                               embedding=embeddings,
#                               url = qdrant_url,
#                               api_key= qdrant_key,
#                               collection_name=f"domino-help")